# Multivariate Kalman filter

# Purpose
* implementation of multivariate kalman filter inspired by: [https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/06-Multivariate-Kalman-Filters.ipynb](https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/06-Multivariate-Kalman-Filters.ipynb)

# Methodology
* Implement a kalman filter to chase a ship at unsteady velocity

# Setup

In [ ]:
# %load imports.py

%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import filterpy.stats as stats

In [ ]:
import math
import numpy as np
from numpy.random import randn

def compute_dog_data(z_var, process_var, count=1, dt=1.):
    "returns track, measurements 1D ndarrays"
    x, vel = 0., 1.
    z_std = math.sqrt(z_var) 
    p_std = math.sqrt(process_var)
    xs, zs = [], []
    for _ in range(count):
        v = vel + (randn() * p_std)
        
        x += v*dt        
        xs.append([x,v])
        
        zs.append(x + randn() * z_std)  
        
    return np.array(xs), np.array(zs), 

In [ ]:
from filterpy.common import Q_discrete_white_noise

dt = 1.
R_var = 10
Q_var = 0.01

count = 50
track, zs = compute_dog_data(R_var, Q_var, count)


In [ ]:
from scipy.linalg import inv

def filter(x,P,F,H,R,Q):

    xs, cov = [], []
    for z in zs:
        # predict
        x = F @ x
        P = F @ P @ F.T + Q
        
        #update
        S = H @ P @ H.T + R
        K = P @ H.T @ inv(S)
        y = z - H @ x
        x += K @ y
        P = P - K @ H @ P
        
        xs.append(x)
        cov.append(P)
    
    xs, cov = np.array(xs), np.array(cov)
    return xs, cov

In [ ]:
x = np.array([[10.0, 4.5]]).T  # state mean
P = np.diag([500, 49])         # state covariance
F = np.array([[1, dt],         # transition function
              [0,  1]])
H = np.array([[1., 0.]])       # measurement function
R = np.array([[R_var]])        # measurement covariance
# z : measurement mean
Q = Q_discrete_white_noise(dim=2, dt=dt, var=Q_var)  # process covariance

xs,cov=filter(x=x, P=P, F=F, H=H, R=R, Q=Q)

In [ ]:
fig,ax=plt.subplots()

ax.plot(track[:,0], label='real')
ax.plot(xs[:,0], label='filter')
ax.set_ylabel('x [m]')
ax.legend()

fig,ax=plt.subplots()

ax.plot(track[:,1], label='real')
ax.plot(xs[:,1], label='filter')
ax.set_ylabel('v [m/s]')
ax.legend()

In [ ]:
from scipy.stats import multivariate_normal

ncols = 4
fig,axes=plt.subplots(ncols=ncols)
fig.set_size_inches(15,5)

for n,i in enumerate(range(0, len(xs), int(len(xs)/(ncols-1)))):

    mus = xs[i].flatten()
    rv = multivariate_normal(mean=mus, cov=cov[i])
    
    x_ = np.linspace(np.min(xs[:,0]), np.max(xs[:,0]),50)
    y_ = np.linspace(np.min(xs[:,1]), np.max(xs[:,1]),50)
    
    x, y = np.meshgrid(x_, y_)
    pos = np.dstack((x, y))
    
    ax=axes[n]
    ax.contourf(x, y, rv.pdf(pos));